In [11]:
import numpy as np
import pickle

with open(".state.pkl", "rb") as f:
    status = pickle.load(f)

print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-amr-lq") and x.endswith("_separate")])
print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-amr-lq") and x.endswith("_whole")])
print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-amr-hq") and x.endswith("_separate")])
print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-amr-hq") and x.endswith("_whole")])
print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-gsm-fr") and x.endswith("_separate")])
print([np.mean(v["PESQ"]) for x, v in status.items() if x.startswith("0-gsm-fr") and x.endswith("_whole")])

[1.7123625, 1.9748999999999994, 1.9290625, 1.7942500000000003, 1.8229125]
[1.6608875, 1.8495624999999998, 1.8651624999999998, 1.8772125000000002, 1.9100375]
[1.8038874999999996, 2.2275875, 2.2032499999999997, 2.095225, 2.09445]
[1.7506999999999997, 2.0308124999999997, 2.0418250000000002, 2.1175249999999997, 2.1191375]
[1.7149250000000003, 2.1972625, 2.2316375, 2.0488, 2.0229500000000002]
[1.6268, 2.0752249999999997, 1.998525, 2.0590125, 1.9749000000000003]


In [12]:
best = [[v["name"] for x, v in status.items() if x.startswith("0-amr-lq") and x.endswith("_separate")][1],
         [v["name"] for x, v in status.items() if x.startswith("0-amr-lq") and x.endswith("_whole")][-1],
         [v["name"] for x, v in status.items() if x.startswith("0-amr-hq") and x.endswith("_separate")][1],
         [v["name"] for x, v in status.items() if x.startswith("0-amr-hq") and x.endswith("_whole")][-1],
         [v["name"] for x, v in status.items() if x.startswith("0-gsm-fr") and x.endswith("_separate")][2],
         [v["name"] for x, v in status.items() if x.startswith("0-gsm-fr") and x.endswith("_whole")][1]]
best

['model_2019-04-11T04:37:21.520666.zip',
 'model_2019-04-13T07:19:54.394461.zip',
 'model_2019-04-13T23:28:48.388411.zip',
 'model_2019-04-16T04:38:40.812970.zip',
 'model_2019-04-17T00:50:08.300275.zip',
 'model_2019-04-16T22:41:28.735783.zip']

In [13]:
import sear_lib
import tqdm

models = [x.replace("model_", "") for x in tqdm.tqdm(best)]
models = [sear_lib.ModelInstance.load("runs/" + x) for x in tqdm.tqdm(models)]

100%|██████████| 6/6 [00:46<00:00,  8.01s/it]


In [14]:
models

In [ ]:
from copy import copy
import keras
import keras.backend as K
import os

codecs = ["amr-lq", "amr-hq", "gsm-fr"]
noise_snr = [0]

evaluations = {}
index = 0

for ix, config in enumerate([sear_lib.DatasetPreparation.setting_for(codec, snr, training_data=0) for codec in codecs for snr in noise_snr]):
    # config.run()
    for baseline_mdl in models[2 * ix : 2 * ix + 2]:
        mdl = copy(baseline_mdl)
        new_out = keras.layers.Lambda(lambda x: K.concatenate([
            x[0],
            x[1][:, :, 129:]
        ]), name="my_lambda")([mdl.model.input, mdl.model.output])
        model = keras.models.Model(mdl.model.input, new_out)
        model.summary()
        mdl.model = model
        mdl.name = str(index)
        index += 1
        eval_task = sear_lib.Evaluation(mdl)
        evaluation = eval_task.run()
        evaluations[index] = evaluation
        mdl = copy(baseline_mdl)
        new_out = keras.layers.Lambda(lambda x:
            x * K.concatenate([
            K.constant(1, shape=[129]),
            K.constant(0, shape=[128]),
        ]), name='mask_output')(mdl.model.output)
        model = keras.models.Model(mdl.model.input, new_out)
        model.summary()
        mdl.model = model
        mdl.name = str(index)
        index += 1
        eval_task = sear_lib.Evaluation(mdl)
        evaluation = eval_task.run()
        evaluations[index] = evaluation
        
evaluations

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_lf (InputLayer)           (None, None, 129)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 129)    0           input_lf[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, 129, 1) 0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, 129, 10 60          lambda_2[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (

Model name: 0, MSE: 2.8254685401916504 +- 0.6713334321975708, MAE: 2.0837650299072266 +- 0.5900295972824097, PESQ: 1.7204500000000003 +- 0.4137721262965885, LSD: 15.766657123188029 +- 6.1999475184976065
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_lf (InputLayer)           (None, None, 129)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 129)    0           input_lf[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, 129, 1) 0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2

Model name: 1, MSE: 15.023162841796875 +- 1.7800004482269287, MAE: 10.909238815307617 +- 1.9731333255767822, PESQ: 1.1425999999999998 +- 0.27446696522532543, LSD: 70.49906598645953 +- 7.944288022665937
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_lf (InputLayer)           (None, None, 129)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 129)    0           input_lf[0][0]                   
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, 129, 1) 0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d

Model name: 2, MSE: 2.851353883743286 +- 0.6483910083770752, MAE: 2.116473436355591 +- 0.5767190456390381, PESQ: 1.6848500000000002 +- 0.3968054025589874, LSD: 15.369747697807485 +- 5.919852048194127
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_lf (InputLayer)        (None, None, 129)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, None, 129)         0         
_________________________________________________________________
lambda_2 (Lambda)            (None, None, 129, 1)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, 129, 10)     60        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, None, 129, 10)     0         
_________________________________________________________________
conv2d_2

In [15]:
[(np.mean(x['PESQ']), np.std(x['PESQ'])) for x in evaluations.values()]

[(1.7204500000000003, 0.4137721262965885),
 (1.1425999999999998, 0.27446696522532543),
 (1.6848500000000002, 0.3968054025589874),
 (1.1076875, 0.16902777831986668),
 (1.7250374999999998, 0.4190946326234088),
 (1.1044375, 0.16492906382366332),
 (1.7063000000000001, 0.3959450214360575),
 (1.1399124999999999, 0.23624649170675532),
 (1.7166000000000001, 0.4234410702801512),
 (1.128425, 0.2480956859258137),
 (1.7236624999999997, 0.42679406461869873),
 (1.1387875, 0.25656333008391907)]